После загрузки изображений с сайта Кинопоиск получены три папки: **sample_dataset**, в которой хранятся единичные качественные изображения знаменитостей, которые могут быть использованы в качестве эталонных; **gallery**, в которой могут находится до пяти дополнительных изображений; **images**, в которой также содержится некоторое количество дополнительных изображений. 

Дополнительные изображения разного качества. Для очень популярных знаменитостей их количество в папке **images** может достигать нескольких сотен. Поэтому, целесообразно отобрать из них наиболее релевантные изображения лица для каждой знаменитости.

Для этого создадим фнукцию, которая возьмёт список знаменитостей из папки **sample_dataset** и, сохрнаяя разеление знаменитостей на отечественных и иностранных, получит доступные дополнительные изображения для каждой знаменитости. Для каждого дополнительного изображения будет вычислен эмбединг на основе библиотеки *face_recognition* и сопоставлен с эмбедином эталонных изображений. По косинусному расстоянию между эмбедингом эталонного изображения и эмбедигами остальных изображений будет произведён отбор наиболее релевантных изображений.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import notebook
import face_recognition
import pandas as pd
import numpy as np
import pickle
import shutil
import os, sys
import glob
import math

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
from PIL import UnidentifiedImageError

In [ ]:
# функция отбора изображений для обучающего датасета

def selection_images(gender='men'):
    pet_path = './Documents/DataScience/PET-project'
    
    # Создание папки под финальный датасет
    if os.path.exists(f'{pet_path}/train') != True:
        os.mkdir(f'{pet_path}/train')
    
    # Создание подпапок для актеров и актрис
    if os.path.exists(f'{pet_path}/train/{gender}') != True:
        os.mkdir(f'{pet_path}/train/{gender}')
    
    nation = ['eng', 'rus']
    for natio in nation:
        # Создание подпапок для разделения знаменитостей на иностранных и отечественных
        if os.path.exists(f'{pet_path}/train/{gender}/{natio}') != True:
            os.mkdir(f'{pet_path}/train/{gender}/{natio}')
        
        # Получаем список знаменитостей из папки с эталонными изображениями
        folders = glob.glob(f'{pet_path}/sample_dataset/{gender}/{natio}/*')
                            
        # Производим перебор списка знаменитостей с отбором наилучших изображений
        for folder in notebook.tqdm(folders):
            if natio == 'eng':
                name = folder[folder.find('eng\\') + 4:].strip()
            elif natio == 'rus':
                name = folder[folder.find('rus\\') + 4:].strip()
            
            if os.path.exists(f'{pet_path}/train/{gender}/{natio}/{name}') != True:
                os.mkdir(f'{pet_path}/train/{gender}/{natio}/{name}')
                print(f'Directory for {name} created!')
            else:
                print(f'Images for {name} already uploaded!')
                continue
            
            # Путь к эталонным изображениям
            path_to_sample = f'{pet_path}/sample_dataset'
            
            # Путь к галерее дополнтельных изображений знаменитостей
            path_to_gallery = f'{pet_path}/gallery'
            
            # Путь к расширенному набору изображений знаменитостей
            path_to_images = f'{pet_path}/images'
            
            # Формируем список файлов для последующего отбора
            sample_file = os.listdir(f'{path_to_sample}/{gender}/{natio}/{name}')
            files =  [f'{path_to_sample}/{gender}/{natio}/{name}/' + file for file in sample_file]
            
            try:
                gallery_files = os.listdir(f'{path_to_gallery}/{gender}/{natio}/{name}')
                [files.append(f'{path_to_gallery}/{gender}/{natio}/{name}/' + file) for file in gallery_files if \
                os.stat(f'{path_to_gallery}/{gender}/{natio}/{name}/' + file).st_size > 0]
                
                images_files = os.listdir(f'{path_to_images}/{gender}/{natio}/{name}')
                [files.append(f'{path_to_images}/{gender}/{natio}/{name}/' + file) for file in images_files if \
                os.stat(f'{path_to_images}/{gender}/{natio}/{name}/' + file).st_size > 0]
            except FileNotFoundError:
                try:
                    print(f'Path :: {path_to_gallery}/{gender}/{natio}/{name} - not found!')
                    shutil.copy(f'{path_to_sample}/{gender}/{natio}/{name}/256.jpg',\
                                f'{pet_path}/train/{gender}/{natio}/{name}/Image_0.jpg')
                except FileNotFoundError:
                    print(f'File :: {path_to_sample}/{gender}/{natio}/{name}/256.jpg - not found!')
                    continue
                continue
            
            # Вычисляем эмбединги для лиц на каждом изображении
            embedings = []
            for file in files:
                try:
                    sample_face = face_recognition.load_image_file(file)
                    embedings.append(face_recognition.face_encodings(sample_face)[0])
                except (IndexError, UnidentifiedImageError):
                    print(f'Not found face in {file}')
                    continue
            
            if len(embedings) > 1:
                # Группировка изображений по кластерам для последующего отбора
                agg = AgglomerativeClustering(n_clusters=None, distance_threshold=0.1,\
                                              affinity='cosine', linkage='complete').fit(embedings)
                
                # Сохраняем в переменных полученные лейблы и количество классов
                labels = agg.labels_
                n_classes = len(set(labels))
                
                # Разбираем файлы по полученным группам
                files_by_group = [[] for x in range(n_classes)]
                
                for x in range(n_classes):
                    for j in range(len(labels)):
                        if labels[j] == x:
                            files_by_group[x].append(files[j])
                            
                # Определяем самый большой кластер
                max_labels_count = 0
                
                for x in range(1, n_classes):
                    if len(files_by_group[x]) > len(files_by_group[max_labels_count]):
                        max_labels_count = x
                        
                # Находим в какой кластер попало эталонное изображение
                group_with_sample = 0
                
                for x in range(n_classes):
                    if f'{path_to_sample}/{gender}/{natio}/{name}/256.jpg' in files_by_group[x]:
                        group_with_sample = x
                        break
                        
                # Формируем словарь с ключами в виде имени файла и значениями в виде эмбедингов
                dictionary = dict(zip(files, embedings))
                
                # Проверяем совпадает ли номер наибольшего кластера с номером кластера, в котором обнаружено эталонное 
                # изображение. Если нет - то объединяем эти два кластера.
                if max_labels_count == group_with_sample:
                    main_files = files_by_group[max_labels_count]
                else:
                    main_files = files_by_group[group_with_sample] + files_by_group[max_labels_count]
                    
                # Вычисляем косинусное расстояние, сохраняем его в словарь и по нему отбираем наилучшие файлы как наиболее
                # близкие к эталонному изображению
                rates = dict()
                
                for f in main_files[1:]:
                    sample_key = f'{path_to_sample}/{gender}/{natio}/{name}/256.jpg'
                    cos = cosine_similarity(dictionary[sample_key].reshape(1, -1), dictionary[f].reshape(1, -1))[0][0]
                    rates[cos] = f
                    
                sorted_rates = list(rates.keys())
                sorted_rates.sort(reverse=True)
                
                best_images = [rates[x] for x in sorted_rates if 0.99 > x > 0.93]
                
                # Рекурсивная функция для удаления дубликатов из отобранных изображений
                def doubles_cleaner(images, pos, dictionary=dictionary):
                    if pos == len(images)-1:
                        return(images)
                    else:
                        for x in range(pos+1, len(images)):
                            if cosine_similarity(dictionary[images[pos]].reshape(1, -1),\
                                                 dictionary[images[x]].reshape(1, -1))[0] > 0.98:
                                del images[x]
                                return doubles_cleaner(images, pos=pos+1)
                            return doubles_cleaner(images, pos=pos+1)
                        
                try:
                    doubles_cleaner(best_images, pos=0)
                except:
                    pass
                
                # Добавляем в начало списка отобранных файлов эталонное изображение
                best_images.insert(0, sample_key)
                
                # сохранение полученных изображений в итоговый тренировочный датасет
                i = 0
                for best_image in best_images:
                    shutil.copy(best_image, f'{pet_path}/train/{gender}/{natio}/{name}/Image_{i}.jpg')
                    i += 1
            else:
                try:
                    shutil.copy(f'{path_to_sample}/{gender}/{natio}/{name}/256.jpg',\
                                f'{pet_path}/train/{gender}/{natio}/{name}/Image_0.jpg')
                except FileNotFoundError:
                    print(f'File :: {path_to_sample}/{gender}/{natio}/{name}/256.jpg - not found!')
                    continue

In [ ]:
selection_images(gender='women')

In [ ]:
selection_images()

Изображения отобраны. Визуальный анализ сохранённых изображений показал, что разброс количества фотографий лиц для отдельных знаменитостей все ещё велик и находится в диапазоне от ноля до полутора сотен. Поэтому, чтобы избежать такого дисбаланса, произведём следующее:
* удалим всех знаменитостей, у которых число изображений менее двух;
* ограничим максимальное число изображений 25 фотографими лиц для каждой знаменитости, удалив лишние в конце списка.

Напишем для этого соотвествующую функцию.

In [ ]:
# функция для удаления лишних изображений

def deleting_excess_images(gender='men'):
    train_path = './Documents/DataScience/PET-project'
    
    nation = ['eng', 'rus']
    for natio in nation:
        # Получаем список знаменитостей
        folders = glob.glob(f'{train_path}/train/{gender}/{natio}/*')
        
        # Производим перебор списка знаменитостей
        for folder in notebook.tqdm(folders):
            if natio == 'eng':
                name = folder[folder.find('eng\\') + 4:].strip()
            elif natio == 'rus':
                name = folder[folder.find('rus\\') + 4:].strip()
                
            # Получаем список изображений для каждой знаменитости
            try:
                files = glob.glob(f'{train_path}/train/{gender}/{natio}/{name}/*')
                
                # Оставим только папки с количеством изображений 2 и более, а также если файлов больше 25 в папке, 
                # о оставляем только 25, первых, а остальные удаляем
                if len(files) < 2:
                    shutil.rmtree(f'{train_path}/train/{gender}/{natio}/{name}/', ignore_errors=True)
                    print(f'Directory :: {train_path}/train/{gender}/{natio}/{name}/ contains less that 2 images and deleted!')
                elif len(files) > 25:
                    # Получаем уникальные номера изображений в папках знаменитостей
                    file_numbers = []
                    for file in files:
                        f_name = file.split('\\')[-1]
                        file_numbers.append(int(f_name.replace('Image_', '').replace('.jpg', '')))
                        
                    # Сортировка списка
                    file_numbers.sort()
                    
                    # Получаем идентификаторы файлов для удаления
                    del_files = file_numbers[25:]

                    # Удаляем файлы
                    for num in del_files:
                         os.remove(f'{train_path}/train/{gender}/{natio}/{name}/Image_{str(num)}.jpg')
                    print(f'Excess images in folder :: {train_path}/train/{gender}/{natio}/{name} - successful deleted!')
            except FileNotFoundError:
                print(f'Path :: {train_path}/train/{gender}/{natio}/{name} - not found!')
                continue

In [ ]:
deleting_excess_images()

In [ ]:
deleting_excess_images(gender='women')

На этом подготовку обучающих датсетов можно считать завершенной и перейти к обучению моделей.